# NOT WORKING 

In [1]:
#!pip install onnx
#!pip install onnx_tf
# pip install -e . https://github.com/onnx/onnx-tensorflow

In [1]:
import onnx
from onnx_tf.backend import prepare # install from master
import tensorflow as tf
import numpy

tf.__version__ , onnx.__version__, numpy.__version__



/home/ale/Dropbox/projects/onnx-tensorflow/onnx_tf/common/__init__.py:87: UserWarning: onnx_tf.common.get_outputs_names is deprecated. It will be removed in future release. Use TensorflowGraph.get_outputs_names instead.
  warnings.warn(message)



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



/home/ale/Dropbox/projects/onnx-tensorflow/onnx_tf/common/__init__.py:87: UserWarning: FrontendHandler.get_outputs_names is deprecated. It will be removed in future release. Use node.outputs instead.
  warnings.warn(message)


('1.13.1', '1.5.0', '1.16.3')

In [3]:
onnx_model = onnx.load("huia2.onnx")  # load onnx model
#output = prepare(onnx_model).run(input)  # run the loaded model

In [4]:
#tf_rep.export_graph('huia_tensor_gpu.pb')

In [ ]:
tensorflowjs_converter \
    --input_format=tf_frozen_model \
    --output_node_names='MobilenetV1/Predictions/Reshape_1' \
    /huia_tensor.pb \
    /huia_tensor_js

In [11]:
import numpy as np
# convert array order from pytorch to tensorflow
img_raw = tf.io.read_file('./training_data/images/backpack_107.png')
with tf.Session().as_default():
    image = tf.io.decode_image(img_raw,channels=3).eval()
    resized_image = tf.image.resize_images(image, [224, 224]).eval()

#normalize image
#offset = 127.5;
#resized_image = (resized_image-offset)/offset  # convert to -1 : 1 


imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
mean = imagenet_stats[0]
std = imagenet_stats[1]

def normalize(x, mean,std):
    for i in range(0,2):
        x[i] = x[i]-mean[i]/std[i]    
    return x

print(resized_image/255)

norm_image = normalize(resized_image/255,mean,std)


# transpose image
trans_image =  np.transpose(norm_image, (2, 0, 1))
trans_image.shape

# create extra dimension
exp_img = np.expand_dims(trans_image, axis=0)
exp_img.shape

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


(1, 3, 224, 224)

In [7]:
POSE_CLASSES =['backpack',
 'dramatic',
 'fly',
 'hadouken',
 'moonwalk',
 'normal',
 'underarm',
 'wings']

In [8]:
tf_rep = prepare(onnx_model,device='CPU')


/home/ale/Dropbox/projects/onnx-tensorflow/onnx_tf/common/handler_helper.py:73: UserWarning: Unknown op ConstantFill in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/home/ale/Dropbox/projects/onnx-tensorflow/onnx_tf/common/handler_helper.py:73: UserWarning: Unknown op DynamicSlice in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/home/ale/Dropbox/projects/onnx-tensorflow/onnx_tf/common/handler_helper.py:73: UserWarning: Unknown op ImageScaler in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/home/ale/Dropbox/projects/onnx-tensorflow/onnx_tf/common/handler_helper.py:70: UserWarning: Fail to get since_version of ThresholdedRelu in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))


Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


/home/ale/Dropbox/projects/onnx-tensorflow/onnx_tf/handlers/backend/pool_mixin.py:124: UserWarning: Using the pooling op in compatibility mode.This means your graph cannot be serialized.Please configure your pooling operation to only use paddings that correspond to Tensorflow SAME or VALID padding.
  "correspond to Tensorflow SAME or VALID padding.", UserWarning)


Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
Use keras.layers.flatten instead.


In [9]:
output = tf_rep.run(exp_img) #.eval()  # run the loaded model
print(POSE_CLASSES[np.argmax(output)])

normal


In [36]:
output


Outputs(_0=array([[  1.2426745,  14.113052 , -23.66965  ,  44.07164  , -14.209565 ,
         16.602457 , -29.920212 , -47.434143 ]], dtype=float32))